# Subtitle Remover

Goal was to create a routine to remove Hardcoded subtitles in a Video file.
This uses Open-CV-python or Open Computer Vision for Python

In [ ]:
import numpy as np
import cv2

img = cv2.imread("./Sharpe.png")

def hasSubTitle(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mx = np.max(hsv[1])
    return mx >= 255

print(hasSubTitle(img))

## Removing the Subtitile

using a Blur mask and a Fill

In [ ]:
def removeSubTitle(img, t):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray,t,255,cv2.THRESH_BINARY_INV)
    maskblur = cv2.GaussianBlur(thresh, (5,5), 0)
    ret, mask = cv2.threshold(maskblur,t,255,cv2.THRESH_BINARY)
    mask1 = mask
    for i in range(5):
        maskblur = cv2.GaussianBlur(mask, (5,5), 0)
        ret, mask = cv2.threshold(maskblur,t,255,cv2.THRESH_BINARY)

    ret, mask = cv2.threshold(maskblur,t,255,cv2.THRESH_BINARY_INV)
    cv2.imshow("Image", cv2.inpaint(img, mask,3,cv2.INPAINT_TELEA))

img = cv2.imread("./Sharpe.png")
removeSubTitle(img, 240)

## Opening FFMPEG-Pipe

In [ ]:
import subprocess as sp

FFMPEG_BIN = "ffmpeg"

def openRead(input:str):
    command = [ FFMPEG_BIN,
                '-i', input,
                '-f', 'image2pipe',
                '-pix_fmt', 'rgb24',
                '-vcodec', 'rawvideo', '-']
    return sp.Popen(command, stdout = sp.PIPE, bufsize=10**8)

def openWrite(output:str, size:str, fps:str):
    command = [ FFMPEG_BIN,
        '-y', # (optional) overwrite output file if it exists
        '-f', 'rawvideo',
        '-vcodec','rawvideo',
        '-s', size, # size of one frame
        '-pix_fmt', 'rgb24',
        '-r', fps, # frames per second
        '-i', '-', # The imput comes from a pipe
        '-an', # Tells FFMPEG not to expect any audio
        output ]

    print(command)

    return sp.Popen( command, stdin=sp.PIPE, stderr=sp.PIPE)